# Communication, RS-232, i2c and all that. 

## (Controlling an interface remotely with software).

Microcontrollers are great for collecting data, controller motors or power sources, but when it comes to managing very complex tasks, or large amounts of data they are not optimal. For isolated systems one solution to this problem is to store data to non-volitile memory (e.g., SD card) and recover the data later for analysis. However often we can connect the microcontroller to a more sophisticated system to manage and retrieve data in real time.